In [1]:
import pandas as pd
import numpy as np
import itertools
from scipy.spatial import distance

pd.set_option('display.max_rows', 500)

### Create inputs

### Populate dataframe

In [2]:
def fill_dataframe(objects, objects_at_once=2):
    df = pd.DataFrame(columns=['from_node', 'to_node', 'obj_at_once', 'dist', 'weight_new'])
    # Generate list of combinations w/o duplicates
    nodes = list(itertools.chain(*[itertools.combinations(objects, i+1) for i in range(len(objects))]))
    nodes = [''.join(node) for node in nodes]
    
    node_list = []
    for x in range(0, len(nodes)):
        if len(nodes[x]) <= 2:
            df.loc[x, 'from_node'] = 'z'
            df.loc[x, 'to_node'] = nodes[x]
            df.loc[x, 'obj_at_once'] = len(nodes[x])
    
        for y in range(x, len(nodes)):
            if 1 <= (len(nodes[y]) - len(nodes[x])) <= 2:
                if all(x in nodes[y] for x in nodes[x]):
                    node_list.append((nodes[y], nodes[x]))
    
    for tup in node_list:
        df = df.append({'from_node': tup[1], 'to_node': tup[0], 'obj_at_once': len(tup[0]) - len(tup[1])}, ignore_index=True)
    
    # No constraints for how many objects at once
    if objects_at_once == 2:
        df = df
    elif objects_at_once == 1:
        # Add constraint: Only 1 object at once
        df = df.loc[df['obj_at_once'] == 1]

    # Reset index
    df = df.reset_index(drop=True)
    
    return df

### Calculate distances

In [3]:
def calculate_distances(data):
    for row in range(0, len(data)):
        # Distance = start -> obj 1 -> table
        if data['from_node'][row] == 'z' and len(data['to_node'][row]) == 1:
            data.loc[row, 'dist'] = (distance.euclidean(
                        coordinates['start'], 
                        coordinates[data['to_node'][row]]) +
                    distance.euclidean(
                        coordinates[data['to_node'][row]], 
                        coordinates['table'])
                    )
    
        # Distance = start -> obj 1 -> obj 2 -> table
        elif data['from_node'][row] == 'z' and len(data['to_node'][row]) != 1:
            data.loc[row, 'dist'] = (distance.euclidean(
                        coordinates['start'], 
                        coordinates[data['to_node'][row][0]]) +
                    distance.euclidean(
                        coordinates[data['to_node'][row][0]], 
                        coordinates[data['to_node'][row][1]]) +
                    distance.euclidean(
                        coordinates[data['to_node'][row][1]], 
                        coordinates['table'])
                    )
    # Distance = table -> obj 1 -> (obj 2) -> table
        else:
            # Get difference between sequences (from_node, to_node)
            diff = [x for x in data['to_node'][row] if x not in data['from_node'][row]]
        
            if len(diff) == 1:
                data.loc[row, 'dist'] = distance.euclidean(
                        coordinates['table'], 
                        coordinates[diff[0]]) * 2
        
            elif len(diff) == 2:
                data.loc[row, 'dist'] = (distance.euclidean(
                        coordinates['table'], 
                        coordinates[diff[0]]) +
                    distance.euclidean(
                        coordinates[diff[0]], 
                        coordinates[diff[1]]) +
                    distance.euclidean(
                        coordinates[diff[1]], 
                        coordinates['table'])
                    )
    return data

In [54]:
def calculate_edge_weights_params(data, objects, c, k):
    # Reset weights according to weight parameters
    for row in range(0, len(data)):
        diff = [x for x in data['to_node'][row] if x not in data['from_node'][row]]
        
        for obj in objects:
            if len(diff) == 1 and obj in diff:
                data.loc[row, 'weight_new'] = (data['dist'][row] ** k[diff[0]]) * c[diff[0]]
                
            elif len(diff) == 2 and obj in diff:
                data.loc[row, 'weight_new'] = (data['dist'][row] ** k[diff[0]]) * c[diff[0]]
                data.loc[row, 'weight_new'] = (data['weight_new'][row] ** k[diff[1]] * c[diff[0]])
                
    return data

In [51]:
objects = ['t', 'n', 's', 'p', 'c']

coordinates = {'c': (1,4),
              'n': (1,1),
              'p': (1,4),
              's': (1,2),
              't': (1,1),
              'start': (2,2),
              'table': (4,3)}

c1 = {'c': 1.2,
    'n': 1.0,
    'p': 1.2,
    's': 1.2,
    't': 1.0}

k1 = {'c': 1.0,
    'n': 0.85,
    'p': 0.8,
    's': 1.0,
    't': 0.75}

c0 = {'c': 1.0,
    'n': 1.0,
    'p': 1.0,
    's': 1.0,
    't': 1.0}

k0 = {'c': 1.0,
    'n': 1.0,
    'p': 1.0,
    's': 1.0,
    't': 1.0}

In [29]:
def predict_sequence(data, objects, coordinates):
    prediction = []
    possible_items = dict.fromkeys(objects, 0)
    
    for row in range(0, len(data)):
        for obj in objects:
            if data.loc[row, 'to_node'] == obj:
                possible_items[obj] = data.loc[row, 'weight_new']
                
    first_minval = min(possible_items.values())
    first_minval = [k for k, v in possible_items.items() if v==first_minval]
    
    print(possible_items)
    prediction.append(first_minval[0])
    del possible_items[first_minval[0]]
    
    while bool(possible_items) == True:
        for row in range(0, len(data)):
            for obj in possible_items.keys():
                if data.loc[row, 'to_node'] == obj:
                    possible_items[obj] = data.loc[row, 'weight_new']
                    
        minval = min(possible_items.values())
        minval = [k for k, v in possible_items.items() if v==minval]
        print(possible_items)
        prediction.append(minval[0])
        del possible_items[minval[0]]
    
    return prediction

In [75]:
data2 = fill_dataframe(objects, objects_at_once=1)
data2 = calculate_distances(data2)
data2 = calculate_edge_weights_params(data2, objects, c0, k0)
#data2

In [76]:
predict_sequence(data2, objects,coordinates)

{'t': 5.0197648378370845, 'n': 5.0197648378370845, 's': 4.16227766016838, 'p': 5.39834563766817, 'c': 5.39834563766817}
{'t': 5.0197648378370845, 'n': 5.0197648378370845, 'p': 5.39834563766817, 'c': 5.39834563766817}
{'n': 5.0197648378370845, 'p': 5.39834563766817, 'c': 5.39834563766817}
{'p': 5.39834563766817, 'c': 5.39834563766817}
{'c': 5.39834563766817}


['s', 't', 'n', 'p', 'c']